### Reformat LPR behavioral data to have t0-t239 or t0-t14

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, random, time

import warnings
warnings.filterwarnings('ignore')

In [2]:
starting_dir = os.getcwd()
print (starting_dir)

/Users/kimd999/research/script_not_in_dropbox/srpAnalytics/analysis/latest/1_reformat/all_targets/LPR


In [3]:
# mac - phase I & II - LPR - 240 endpoints
complete_input_file_path= '/Users/kimd999/research/projects/toxicity/per_each_data/phase_I_II/input/LPR/after_Lisa_fix/full/240_endpoints/344_zf_LPR_data_phase_1_2_2020JUNE25_updated_plate_id_for_TX_tall_fixed_full_w_240_endpoints.csv'
# -> 196 unique chemical IDs

# mac - phase I & II - LPR - 15 endpoints
#complete_input_file_path= '/Users/kimd999/research/projects/toxicity/per_each_data/phase_I_II/input/LPR/tall/after_Lisa_plate_id_fix/15_endpoints_149_Toxcast_zf_LPR_data_phase_2_2021APR09.csv'
# -> 148 unique chemical IDs

# mac - phase III - full
#complete_input_file_path= '/Users/kimd999/research/projects/toxicity/per_each_data/phase_III/input/original/behavior/LPR/Tanguay_Phase_3_zf_LPR_data_PNNL_2021MAR23.csv'
# -> 240 unique variables and 215 unique chemical IDs

# constance - phase I & II - LPR - 240 endpoints
#complete_input_file_path= '/qfs/people/kimd999/tox/phase_I_II/LPR/input/tall/344_zf_LPR_data_phase_1_2_2020JUNE25_updated_plate_id_for_TX_tall_full_w_240_endpoints.csv'

# constance - phase I & II - LPR - 15 endpoints
#complete_input_file_path= '/qfs/people/kimd999/tox/phase_I_II/LPR/input/tall/344_zf_LPR_data_phase_1_2_2020JUNE25_updated_plate_id_for_TX_tall_full_w_15_endpoints.csv'

# constance - phase III - full
#complete_input_file_path= '/people/kimd999/tox/phase_III/Tanguay_Phase_3_zf_LPR_data_PNNL_2021MAR23.csv'

In [4]:
df_behav = pd.read_csv(complete_input_file_path, header = 0)
df_behav = df_behav.rename({'endpoint': 'variable'}, axis=1)

df_behav['chemical.id'] = df_behav['chemical.id'].astype(str)
# this recasting is needed for "df_select_1846 = df_select.loc[df_select['chemical.id'] == '1846',:]" later

display(df_behav.head())

,chemical.id,conc,plate.id,well,variable,value
0,1030,NaN,NaN,NaN,t0,NaN
1,1030,0.0,9414.0,A06,t0,0.0
2,1030,0.0,9414.0,A12,t0,2.7
3,1030,0.0,9414.0,B06,t0,0.5
4,1030,0.0,9414.0,B12,t0,0.0


In [8]:
# Keep only relevant columns
columns_to_keep = ['chemical.id', 'conc', 'plate.id', 'well', 'variable', 'value']
df_select = df_behav.loc[:,columns_to_keep]

display("number of unique variables:" + str(len(np.unique(df_select['variable']))))
display("number of chemical_id:" + str(len(np.unique(df_select['chemical.id']))))

'number of unique variables:240'

'number of chemical_id:196'

In [6]:
nan = df_select[df_select['value'].isna()]
display(nan.head())

# [phase III] there is no nan in 'chemical.id', 'conc', 'plate.id', 'well', 'variable'

,chemical.id,conc,plate.id,well,variable,value
0,1030,NaN,NaN,NaN,t0,NaN
193,1030,NaN,NaN,NaN,t1,NaN
386,1030,NaN,NaN,NaN,t2,NaN
579,1030,NaN,NaN,NaN,t3,NaN
772,1030,NaN,NaN,NaN,t4,NaN


In [7]:
#display("before dropna, len(behav_select):"+str(len(df_select)))
#df_select = df_select.dropna(how='any')
#display("after dropna,  len(behav_select):"+str(len(df_select)))

#display(df_select.head())

# phase I & II -> dropped some
# phase III    -> dropped many

'before dropna, len(behav_select):11184240'

'after dropna,  len(behav_select):10977439'

,chemical.id,conc,plate.id,well,variable,value
1,1030,0.0,9414.0,A06,t0,0.0
2,1030,0.0,9414.0,A12,t0,2.7
3,1030,0.0,9414.0,B06,t0,0.5
4,1030,0.0,9414.0,B12,t0,0.0
5,1030,0.0,9414.0,C06,t0,0.0


In [ ]:
''' # basic check of variable #
full_devel = "full"
#full_devel = "devel"

if (full_devel == "full"):
    chemical_id_from_here = np.unique(behav_select['chemical.id'])
else: # full_devel = "devel"
    chemical_id_from_here = np.unique([234])
    
for chemical_index in chemical_id_from_here:
    behav_per_chemical = behav_select.loc[behav_select['chemical.id'] == chemical_index,:]
    print("chemical_index:" + str(chemical_index))

    variables = np.unique(behav_per_chemical['variable'])
    print("variables:" + str(variables))
#    variable_splited = variable.split("t")
    
 #   print (min(variable_splited[1]))
    var_len = len(np.unique(behav_per_chemical['variable']))
    if (var_len != 15):
        display("number of variable:" + str(len(np.unique(behav_per_chemical['variable']))))
'''

### Transpose time points 

In [ ]:
start_time = time.time()
       
def reformat(chemical_index, df_select, reformatted):
    df_per_chemical = df_select.loc[df_select['chemical.id'] == chemical_index,:]
    #print (behav_per_chemical)

    # Append chemical_plate_well as a unique identifier
    # takes long time (1 min?)
    df_per_chemical.insert(0, 'chemical_plate_well', df_per_chemical.loc[:,['chemical.id','plate.id', 'well']].apply(lambda x: '_'.join(x.map(str)), axis = 1))
    
    for cpw in np.unique(df_per_chemical.chemical_plate_well):
        #print (str(cpw))
        per_cpw = df_per_chemical.loc[df_per_chemical.chemical_plate_well == cpw,:]
        per_cpw_grouped = per_cpw.groupby(['chemical.id', 'plate.id', 'well'])
        for name, group in per_cpw_grouped:
            concat_this = pd.DataFrame(
                    {
                    'chemical.id': np.unique(per_cpw['chemical.id']),
                    'plate.id': np.unique(per_cpw['plate.id']),
                    'well': np.unique(per_cpw['well']),
                    'chemical_plate_well': np.unique(per_cpw['chemical_plate_well']),
                    'conc': np.unique(per_cpw['conc'])
                    })
            
            # rename endpoint columns if this is for 15 endpoints
            for time_point in np.arange(len(np.unique(group.variable))):
                #print ("np.unique(group.variable):"+str(np.unique(group.variable)))
                if (len(np.unique(group.variable)) == 15):
                    time_point = time_point + 3
                variable = 't'+ str(time_point)
                #print ("\nvariable:"+str(variable))
                concat_this = pd.concat([concat_this, pd.DataFrame({variable: per_cpw.value[per_cpw.variable == variable].values})],axis = 1)
            reformatted = pd.concat([reformatted, concat_this])
    return reformatted
########### end of def reformat(chemical_index, behav_select, reformatted):


reformatted = pd.DataFrame()

full_devel = "full"
#full_devel = "devel"

chemical_id_from_here = np.unique(df_behav['chemical.id'])

if (full_devel == "devel"):
    randomly_chosen = random.sample(set(chemical_id_from_here), 1)
    chemical_id_from_here = []
    for i in range(len(randomly_chosen)):
        chemical_id_from_here.append(randomly_chosen[i])

#chemical_id_from_here = ['1846']
#chemical_id_from_here = ['471']

total_number_of_chemicals_to_processed = len(chemical_id_from_here)
number_of_chemicals_processed = 0

for chemical_index in chemical_id_from_here:
    print("chemical_index:" + str(chemical_index))
    reformatted = reformat(chemical_index, df_select, reformatted)
    
    number_of_chemicals_processed += 1
    print_this = str(number_of_chemicals_processed) + " chemicals processed out of " + str(total_number_of_chemicals_to_processed)
    print(print_this)

end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("transposing time points is done. It took :"+str(time_took)) 
# took 2 minutes for 1 chemical

chemical_index:53
1 chemicals processed out of 196
chemical_index:54
2 chemicals processed out of 196
chemical_index:56
3 chemicals processed out of 196
chemical_index:57
4 chemicals processed out of 196
chemical_index:58
5 chemicals processed out of 196
chemical_index:60
6 chemicals processed out of 196
chemical_index:61
7 chemicals processed out of 196
chemical_index:62
8 chemicals processed out of 196
chemical_index:63
9 chemicals processed out of 196
chemical_index:64
10 chemicals processed out of 196
chemical_index:65
11 chemicals processed out of 196
chemical_index:66
12 chemicals processed out of 196
chemical_index:67
13 chemicals processed out of 196
chemical_index:69
14 chemicals processed out of 196
chemical_index:70
15 chemicals processed out of 196
chemical_index:129
16 chemicals processed out of 196
chemical_index:155
17 chemicals processed out of 196
chemical_index:220
18 chemicals processed out of 196
chemical_index:227
19 chemicals processed out of 196
chemical_index:24

In [12]:
display(reformatted.head())

,chemical.id,plate.id,well,chemical_plate_well,conc,t0,t1,t2,t3,t4,...,t230,t231,t232,t233,t234,t235,t236,t237,t238,t239
0,3005,10591.0,A01,3005_10591.0_A01,25.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,3005,10591.0,A02,3005_10591.0_A02,5.0,1.8,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,3005,10591.0,A03,3005_10591.0_A03,1.0,0.0,0.0,0.0,0.0,0.0,...,15.7,10.9,8.0,13.3,17.0,13.0,11.2,15.0,7.9,0.0
0,3005,10591.0,A04,3005_10591.0_A04,0.5,0.0,0.0,0.0,0.0,0.0,...,26.6,25.4,24.2,25.7,22.0,16.6,27.2,20.1,22.2,0.0
0,3005,10591.0,A05,3005_10591.0_A05,0.1,3.1,5.2,0.0,0.0,0.0,...,13.4,8.6,10.5,19.3,11.0,17.8,10.4,9.4,12.4,0.0


In [14]:
reformatted_data_filename = str(complete_input_file_path[:-4]) + "_wide_t0_t239_" + str(full_devel) + ".csv"
display ("reformatted_data_filename:", reformatted_data_filename)
reformatted.to_csv(reformatted_data_filename,index=False)


'reformatted_data_filename:'

'/Users/kimd999/research/projects/toxicity/per_each_data/phase_I_II/input/LPR/after_Lisa_fix/full/240_endpoints/344_zf_LPR_data_phase_1_2_2020JUNE25_updated_plate_id_for_TX_tall_fixed_full_w_240_endpoints_wide_t0_t239_devel.csv'

In [ ]:
display(len(np.unique(reformatted['chemical.id'])))
#display(len(np.unique(reformatted_w_non_240_endpoints['chemical.id'])))